# The Synthetic Radio Host - Wiki-talks 🎙️

**Winter 30 Hackathon | Task 1**

Convert any Wikipedia article into a natural-sounding 2-minute Hinglish radio conversation using Google Gemini and ElevenLabs V3.

### Features
- Natural Hinglish (Hindi-English) conversations
- Interruptions, filler words, laughter
- Three conversation styles: RJ, Business, Teams

## Setup

### Add API Keys to Colab Secrets
1. Click the **🔑 key icon** in the left sidebar (or go to Runtime > Manage secrets)
2. Add these two secrets:
   - `GEMINI_API_KEY` - Your Google Gemini API key
   - `ELEVEN_API_KEY` - Your ElevenLabs API key
3. Toggle "Notebook access" ON for both

Then run all cells below!

In [ ]:
!pip install google-generativeai wikipedia-api requests

In [ ]:
# Configuration for The Synthetic Radio Host - Wiki-talks

import os

# ElevenLabs V3 Model Configuration
MODEL_ID = "eleven_v3"

# ElevenLabs API Endpoint
ELEVENLABS_BASE_URL = "https://api.elevenlabs.io/v1/text-to-dialogue"

# Speaker Names Mapping for each variant
SPEAKER_NAMES = {
    "RJ": {"Person A": "Ravi", "Person B": "Priya"},
    "Business": {"Person A": "Amit", "Person B": "Neha"},
    "Teams": {"Person A": "Vikram", "Person B": "Anjali"}
}

# Voice Cast Mapping (maps speaker names to ElevenLabs Voice IDs)
# All Person A names use the same voice ID, all Person B names use the same voice ID
# This can be changed per variant later if needed
VOICE_CAST = {
    # RJ variant
    "Ravi": "tJadXrsBtUAO6pGZKpyW",   # Person A voice
    "Priya": "hczKB0VbXLcBTn17ShYS",   # Person B voice
    # Business variant (same voices for now)
    "Amit": "tJadXrsBtUAO6pGZKpyW",    # Person A voice
    "Neha": "hczKB0VbXLcBTn17ShYS",    # Person B voice
    # Teams variant (same voices for now)
    "Vikram": "tJadXrsBtUAO6pGZKpyW",  # Person A voice
    "Anjali": "hczKB0VbXLcBTn17ShYS"   # Person B voice
}

# Conversation Variants with Hinglish-focused System Prompts
# These are prompt templates that will be formatted with speaker names and content
VARIANTS = {
    "RJ": """### PERSONA
You are a Bollywood-style Radio Scriptwriter and Director.
Your expertise lies in creating high-energy, natural-sounding "Hinglish" (Indian English + Hindi) dialogues.
You understand the nuance of spoken Indian audio: fast-paced, overlapping speech, and emotional modulation.

### CONTEXT
We are producing a 2-minute audio segment for a viral podcast/radio show.
The topic is derived from the following source text:
{text}

### TASK
Convert the source text into a natural conversation script in Hinglish. Use the context of the source text to create a natural conversation.
1. **Language:** Use authentic Hinglish. Do not just translate; use sentence structures like "Arre bhai...", "Samjha na?", "Wahi toh problem hai." Example:
   - *Bad:* "I am going to the market. Do you want to join?"
   - *Good:* "I am going to the market, tu chalega?"
2. **Acting Directions:**
   - You MUST use specific [tags] inside the text to direct the voice.
   - Use **[laughing]** or **[giggling]** for humor.
   - Use **[sighs]** or **[clears throat]** for hesitation.
   - Use **[whispers]** for secrets or dramatic effect.
   - Use **[shouting]** for high excitement.
3. **Pacing & Interruptions:**
   - If Speaker B interrupts Speaker A, start Speaker B's line with **[interrupting]**.
   - Keep sentences short. No long monologues.
4. **Names:** Use the names {speaker_a} and {speaker_b} to create a natural conversation. DO NOT come up with new names.

### FORMAT
You must output a **STRICT JSON ARRAY** of objects.
Each object should have the following fields:
- speaker: The name of the speaker
- text: The text of the speaker
Do NOT include markdown formatting (```json).
**Schema:**
[
  {{
    "speaker": "{speaker_a}",
    "text": "..."
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "..."
  }}
]
**Example Output:**
[
  {{
    "speaker": "{speaker_a}",
    "text": "[shouting] Arre {speaker_b}! Did you see the news? [laughing] It's total madness!"
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "[interrupting] [sighs] Bas kar bhai. I knew you would say that. [whispers] But honestly... I agree."
  }}
]

**Target Length:** Approximately {target_words} words total.""",

    "Business": """### PERSONA
You are a Professional Business Content Scriptwriter specializing in Hinglish corporate dialogues.
Your expertise lies in creating professional, informative, and respectful "Hinglish" (Indian English + Hindi) conversations.
You understand the balance between maintaining professionalism and using natural Indian speech patterns.

### CONTEXT
We are producing a 2-minute audio segment for a professional business podcast/discussion.
The topic is derived from the following source text:
{text}

### TASK
Convert the source text into a natural business conversation script in Hinglish. Use the context of the source text to create a natural conversation.
1. **Language:** Use professional Hinglish. Maintain a respectful, informative tone while using natural Hinglish structures. Example:
   - *Bad:* "I think this is a good strategy for our company."
   - *Good:* "Yeh strategy theek lag rahi hai, but we need to consider the market conditions."
2. **Acting Directions:**
   - You MUST use specific [tags] inside the text to direct the voice.
   - Use **[laughing]** or **[laughs]** for light humor (minimal, professional).
   - Use **[sighs]** or **[clears throat]** for thoughtful pauses.
   - Use **[whispers]** sparingly, only for confidential discussions.
   - Avoid [shouting] - keep it professional.
3. **Pacing & Interruptions:**
   - Keep interruptions minimal and professional. If needed, use **[interrupting]** but make it respectful.
   - Keep sentences clear and structured. Allow for thoughtful pauses.
4. **Names:** Use the names {speaker_a} and {speaker_b} to create a natural business conversation.

### FORMAT
You must output a **STRICT JSON ARRAY** of objects.
Each object should have the following fields:
- speaker: The name of the speaker
- text: The text of the speaker
Do NOT include markdown formatting (```json).
**Schema:**
[
  {{
    "speaker": "{speaker_a}",
    "text": "..."
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "..."
  }}
]
**Example Output:**
[
  {{
    "speaker": "{speaker_a}",
    "text": "Achcha {speaker_b}, I think we should discuss the quarterly results. [clears throat] The numbers are quite interesting."
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "Haan, absolutely. Theek hai, let's analyze the key metrics. [sighs] There are some areas we need to focus on."
  }}
]

**Target Length:** Approximately {target_words} words total.""",

    "Teams": """### PERSONA
You are a Screenwriter for a realistic workplace dramedy (like 'Silicon Valley' but set in Bangalore/Gurgaon).
You are writing a script for two software engineers having a quick, informal sync on Microsoft Teams.
Tone: Candid, slightly cynical, exhausted but productive, and very casual.

### CONTEXT
We are simulating a 2-minute "Teams Call" between two colleagues who trust each other.
The topic of discussion is derived from the following source text:
"{text}"

### TASK
Convert the source text into a natural conversation in Hinglish between **{speaker_a}** and **{speaker_b}**.
1. **Language:** Use "Corporate Hinglish." Mix tech jargon with Hindi casualness.
   - *Keywords to use:* "Bandwidth", "Ping", "Prod", "Deploy", "Ticket", "Bhai", "Yaar", "Scene kya hai".
   - *Bad:* "The project deadline is approaching."
   - *Good:* "Yaar, deadline sar pe hai aur code phat raha hai."
2. **Structure:**
   - **Start:** Start with a "mic check" or screen share issue ("Am I audible?", "Screen dikh raha hai?") by only one of the person.
   - **Middle:** Discuss the content honestly. If it's complex, complain about it slightly. If it's good, be skeptical.
   - **End:** MUST end with going back to work ("Chalo, I have a call", "Code push karna hai").
3. **Acting Directions (ElevenLabs V3):**
   - Use **[sighs]** for work fatigue.
   - Use **[clears throat]** before explaining something technical.
   - Use **[interrupting]** to simulate network lag or talking over each other.
   - Use **[whispers]** when gossiping about management/policy.
   - Use **[laughing]** only for cynical jokes.

### FORMAT
You must output a **STRICT JSON ARRAY** of objects.
Do NOT include markdown formatting (```json).

**Schema:**
[
  {{
    "speaker": "{speaker_a}",
    "text": "..."
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "..."
  }}
]

**Example Output:**
[
  {{
    "speaker": "{speaker_a}",
    "text": "Hello? Hello? [clears throat] Am I audible now?"
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "Haan haan, clear hai. Bol. [sighs] What's the issue with the build?"
  }},
  {{
    "speaker": "{speaker_a}",
    "text": "[interrupting] Arre, it's not the build. Did you see this doc? [whispers] Total chaos."
  }},
  {{
    "speaker": "{speaker_b}",
    "text": "Chalo chodo. [clears throat] Let's focus on the Jira ticket. I'll ping you later."
  }}
]

**Target Length:** Approximately {target_words} words total."""
}

In [ ]:
import json
import re
import requests
from google import genai
import wikipediaapi  # Package: wikipedia-api (install via: pip install wikipedia-api)
from typing import List, Dict, Optional, Tuple

class WikiScraper:
    """Handles Wikipedia content extraction with error handling"""

    def __init__(self):
        self.wiki = wikipediaapi.Wikipedia(
            user_agent='wiki-talks/1.0 (https://github.com/purugoyal-ril/wiki-talks)',
            language='en'
        )

    def scrape(self, url: str, mode: str = "fast") -> Tuple[Optional[str], Optional[str]]:
        """
        Scrape Wikipedia content from URL

        Args:
            url: Wikipedia article URL
            mode: "fast" (summary only) or "pro" (sections, capped at 4000 words)

        Returns:
            Tuple of (content, error_message). content is None if error occurred.
        """
        try:
            # Extract page title from URL
            page_title = self._extract_title_from_url(url)
            if not page_title:
                return None, "Invalid Wikipedia URL format"

            # Get page
            page = self.wiki.page(page_title)

            # Check if page exists
            if not page.exists():
                return None, f"Wikipedia page '{page_title}' not found"

            # Handle disambiguation
            if 'disambiguation' in page.title.lower():
                # Auto-select first option
                if page.links:
                    first_link = list(page.links.keys())[0]
                    page = self.wiki.page(first_link)
                    if not page.exists():
                        return None, f"Could not access disambiguation option: {first_link}"

            # Extract content based on mode
            if mode.lower() == "fast":
                content = page.summary
            else:  # pro mode
                content = self._extract_sections(page, max_words=4000)

            if not content or len(content.strip()) < 50:
                return None, "Page content too short or empty"

            return content, None

        except Exception as e:
            # Handle disambiguation and page errors generically
            # Note: wikipediaapi doesn't have an exceptions module, so we catch all exceptions
            # and check error messages to determine the type
            error_str = str(e).lower()
            if 'disambiguation' in error_str or 'ambiguous' in error_str:
                # Try to auto-select first option from links if we have a page object
                try:
                    # Try to get the page again to access links
                    page_title = self._extract_title_from_url(url)
                    if page_title:
                        page = self.wiki.page(page_title)
                        if hasattr(page, 'links') and page.links:
                            first_link = list(page.links.keys())[0]
                            selected_page = self.wiki.page(first_link)
                            if selected_page.exists():
                                content = selected_page.summary if mode.lower() == "fast" else self._extract_sections(selected_page, max_words=4000)
                                return content, None
                except Exception:
                    pass
                return None, f"Disambiguation error: {str(e)}"
            elif 'page' in error_str or 'not found' in error_str:
                return None, f"Page error: {str(e)}"
            else:
                return None, f"Error scraping Wikipedia: {str(e)}"

    def _extract_title_from_url(self, url: str) -> Optional[str]:
        """Extract page title from Wikipedia URL"""
        # Handle various URL formats
        patterns = [
            r'wikipedia\.org/wiki/([^?#]+)',
            r'wikipedia\.org/wiki/([^/?#]+)',
        ]

        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                title = match.group(1)
                # URL decode
                title = title.replace('_', ' ')
                return title

        return None

    def _extract_sections(self, page, max_words: int = 4000) -> str:
        """Extract content from sections, capped at max_words"""
        content_parts = []
        word_count = 0

        # Add summary first
        if page.summary:
            words = page.summary.split()
            if word_count + len(words) <= max_words:
                content_parts.append(page.summary)
                word_count += len(words)

        # Add sections
        for section in page.sections:
            if word_count >= max_words:
                break

            section_title = section.title
            section_text = section.text
            words = section_text.split()

            if word_count + len(words) <= max_words:
                content_parts.append(f"\n\n## {section_title}\n\n{section_text}")
                word_count += len(words)
            else:
                # Add partial section
                remaining_words = max_words - word_count
                partial_text = ' '.join(words[:remaining_words])
                content_parts.append(f"\n\n## {section_title}\n\n{partial_text}")
                break

        return '\n'.join(content_parts)


class ScriptGenerator:
    """Generates Hinglish conversation scripts using Google Gemini"""

    def __init__(self, api_key: str):
        """
        Initialize ScriptGenerator with Gemini API key

        Args:
            api_key: Google Gemini API key
        """
        self.client = genai.Client(api_key=api_key)
        self.model_name = 'gemini-2.5-flash'
        # Store generation config for use in generate_content
        self.generation_config = {
            "response_mime_type": "application/json",
            "temperature": 0.8
        }

    def generate_script(self, text: str, variant: str = "RJ", duration: int = 120) -> Tuple[Optional[List[Dict]], Optional[str]]:
        """
        Generate Hinglish conversation script from Wikipedia content

        Args:
            text: Wikipedia content text
            variant: "RJ", "Business", or "Teams"
            duration: Target duration in seconds (default 120 for 2 minutes)

        Returns:
            Tuple of (script_json, error_message). script_json is None if error occurred.
        """
        try:
            # Get variant-specific prompt template
            prompt_template = VARIANTS.get(variant, VARIANTS["RJ"])
            
            # Get speaker names for this variant
            speaker_names = SPEAKER_NAMES.get(variant, SPEAKER_NAMES["RJ"])
            speaker_a = speaker_names["Person A"]
            speaker_b = speaker_names["Person B"]
            
            # Calculate target word count (~150 WPM for conversational)
            target_words = int((duration / 60) * 150)  # ~300 words for 2 minutes
            
            # Format the prompt template with actual values
            # Limit text to avoid token limits
            formatted_prompt = prompt_template.format(
                text=text[:3000],
                speaker_a=speaker_a,
                speaker_b=speaker_b,
                target_words=target_words
            )
            
            # Generate script
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=formatted_prompt,
                config=self.generation_config
            )
            
            # Extract JSON from response
            script_text = response.text
            
            # Strip markdown code fences if present
            script_text = self._strip_markdown(script_text)
            
            # Parse JSON
            script_json = json.loads(script_text)
            
            # Validate structure
            if not isinstance(script_json, list):
                return None, "Script must be a JSON array"
            
            # Get expected speaker names for validation
            expected_speakers = [speaker_a, speaker_b]
            
            # Validate each entry
            for entry in script_json:
                if not isinstance(entry, dict):
                    return None, "Each script entry must be a dictionary"
                if "speaker" not in entry or "text" not in entry:
                    return None, "Each entry must have 'speaker' and 'text' fields"
                if entry["speaker"] not in expected_speakers:
                    return None, f"Speaker must be '{speaker_a}' or '{speaker_b}' for variant '{variant}', got: {entry['speaker']}"
            
            return script_json, None
            
        except json.JSONDecodeError as e:
            return None, f"JSON parsing error: {str(e)}"
        except Exception as e:
            return None, f"Error generating script: {str(e)}"

    def _strip_markdown(self, text: str) -> str:
        """Strip markdown code fences from JSON response"""
        # Remove ```json and ``` markers
        text = re.sub(r'^```json\s*', '', text, flags=re.MULTILINE)
        text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
        text = re.sub(r'```$', '', text, flags=re.MULTILINE)
        return text.strip()


class AudioEngine:
    """Generates audio using ElevenLabs V3 Dialogue API"""

    def __init__(self):
        """Initialize AudioEngine"""
        pass

    def generate_dialogue_v3(self, script_json: List[Dict], api_key: str, base_url: Optional[str] = None) -> Tuple[Optional[bytes], Optional[str]]:
        """
        Generate audio using ElevenLabs V3 text-to-dialogue endpoint

        Args:
            script_json: List of dicts with "speaker" and "text" keys
            api_key: ElevenLabs API key
            base_url: Optional custom base URL (defaults to ELEVENLABS_BASE_URL)

        Returns:
            Tuple of (audio_bytes, error_message). audio_bytes is None if error occurred.
        """
        try:
            # Use provided base_url or default from config
            url = base_url or ELEVENLABS_BASE_URL

            # Build dialogue_inputs array
            dialogue_inputs = []

            for line in script_json:
                speaker = line.get("speaker", "Host")
                text = line.get("text", "")

                # Look up voice_id from VOICE_CAST
                voice_id = VOICE_CAST.get(speaker)
                if not voice_id:
                    return None, f"Voice ID not found for speaker: {speaker}"

                dialogue_inputs.append({
                    "text": text,
                    "voice_id": voice_id
                })

            # Prepare API request
            headers = {
                "xi-api-key": api_key,
                "Content-Type": "application/json"
            }

            body = {
                "inputs": dialogue_inputs,
                "model_id": MODEL_ID
            }

            # Make API call
            response = requests.post(url, json=body, headers=headers, timeout=120)

            # Check response
            if response.status_code != 200:
                error_msg = f"ElevenLabs API error: {response.status_code}"
                try:
                    error_detail = response.json()
                    error_msg += f" - {error_detail}"
                except:
                    error_msg += f" - {response.text[:200]}"
                return None, error_msg

            # Return binary audio content
            audio_bytes = response.content
            return audio_bytes, None

        except requests.exceptions.RequestException as e:
            return None, f"Network error: {str(e)}"
        except Exception as e:
            return None, f"Error generating audio: {str(e)}"


In [ ]:
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("Note: Not running in Colab. Using environment variables for API keys.")

# Cell 3: Configuration and API Keys
def get_colab_api_keys():
    """Get API keys from Colab userdata or environment variables"""
    if IN_COLAB:
        try:
            gemini_key = userdata.get('GEMINI_API_KEY')
            eleven_key = userdata.get('ELEVEN_API_KEY')
            return gemini_key, eleven_key
        except Exception as e:
            print(f"Error getting API keys from Colab userdata: {e}")
            print("\nTo set API keys in Colab:")
            print("1. Go to: Runtime > Manage secrets")
            print("2. Add secrets: GEMINI_API_KEY and ELEVEN_API_KEY")
            return None, None
    else:
        # Local environment: use environment variables
        gemini_key = os.environ.get('GEMINI_API_KEY')
        eleven_key = os.environ.get('ELEVEN_API_KEY')

        if not gemini_key:
            print("⚠️  GEMINI_API_KEY not found in environment variables")
        if not eleven_key:
            print("⚠️  ELEVEN_API_KEY not found in environment variables")

        return gemini_key, eleven_key

# Cell 4: Main Pipeline Function
def generate_wiki_talk(wikipedia_url: str, variant: str = "RJ", mode: str = "fast", output_file: str = "wiki_talk_output.mp3"):
    """
    Complete pipeline: Wikipedia URL → Script → Audio

    Args:
        wikipedia_url: Full Wikipedia article URL
        variant: "RJ", "Business", or "Teams"
        mode: "fast" (summary) or "pro" (sections)
        output_file: Output MP3 filename

    Returns:
        Tuple of (success: bool, message: str, script_json: list, audio_path: str)
    """
    print("=" * 60)
    print("The Synthetic Radio Host - Wiki-talks - Generating Hinglish Conversation")
    print("=" * 60)

    # Get API keys
    gemini_key, eleven_key = get_colab_api_keys()
    if not gemini_key:
        return False, "Gemini API key not found. Please set GEMINI_API_KEY in Colab secrets.", None, None
    if not eleven_key:
        return False, "ElevenLabs API key not found. Please set ELEVEN_API_KEY in Colab secrets.", None, None

    # Step 1: Scrape Wikipedia
    print("\n[1/3] Scraping Wikipedia...")
    scraper = WikiScraper()
    content, error = scraper.scrape(wikipedia_url, mode)
    if error:
        return False, f"Wikipedia scraping failed: {error}", None, None
    print(f"✓ Scraped {len(content)} characters from Wikipedia")

    # Step 2: Generate Script
    print("\n[2/3] Generating Hinglish conversation script...")
    script_gen = ScriptGenerator(gemini_key)
    script_json, error = script_gen.generate_script(content, variant, duration=120)
    if error:
        return False, f"Script generation failed: {error}", None, None
    print(f"✓ Generated script with {len(script_json)} dialogue entries")

    # Display script preview
    print("\nScript Preview:")
    for i, entry in enumerate(script_json[:3], 1):
        print(f"  {i}. {entry['speaker']}: {entry['text'][:80]}...")
    if len(script_json) > 3:
        print(f"  ... and {len(script_json) - 3} more entries")

    # Step 3: Generate Audio
    print("\n[3/3] Generating audio with ElevenLabs V3...")
    audio_engine = AudioEngine()
    audio_bytes, error = audio_engine.generate_dialogue_v3(script_json, eleven_key)
    if error:
        return False, f"Audio generation failed: {error}", script_json, None
    print(f"✓ Generated audio ({len(audio_bytes)} bytes)")

    # Save audio file
    try:
        with open(output_file, 'wb') as f:
            f.write(audio_bytes)
        print(f"\n✓ Audio saved to: {output_file}")
        audio_path = os.path.abspath(output_file)
    except Exception as e:
        return False, f"Error saving audio file: {str(e)}", script_json, None

    print("\n" + "=" * 60)
    print("✓ Success! The Synthetic Radio Host - Wiki-talks generation complete")
    print("=" * 60)

    return True, "Success", script_json, audio_path


## Usage

### Parameters
- **wikipedia_url**: Any Wikipedia article URL
- **variant**:
  - `"RJ"` - Casual radio host style (fun, energetic)
  - `"Business"` - Professional discussion
  - `"Teams"` - Sports-focused, high energy
- **mode**:
  - `"fast"` - Uses article summary only
  - `"pro"` - Uses full sections (up to 4000 words)

### Run the cell below to generate!

In [ ]:
if __name__ == "__main__":
    # Example: Mumbai Indians Wikipedia article
    example_url = "https://en.wikipedia.org/wiki/Mumbai_Indians"

    print("The Synthetic Radio Host - Wiki-talks - Example Run")
    print(f"URL: {example_url}")
    print(f"Variant: RJ")
    print(f"Mode: fast\n")

    success, message, script, audio_path = generate_wiki_talk(
        wikipedia_url=example_url,
        variant="RJ",
        mode="fast",
        output_file="mumbai_indians_talk.mp3"
    )

    if success:
        print(f"\n✓ Script JSON:")
        print(json.dumps(script, indent=2, ensure_ascii=False))
        print(f"\n✓ Audio file: {audio_path}")
        print("\nYou can play the audio file or download it from Colab.")
    else:
        print(f"\n✗ Error: {message}")